In [4]:
# 사진 갯수
from glob import glob
pngs = sorted(glob('unzip/*/*/*/*/*.png')) 
len(pngs)

473351

In [10]:
# 파일명 파싱
from tqdm.auto import tqdm
from pathlib import Path
pngs = [Path(e).stem for e in pngs]
pngs = [e.split('_') for e in pngs]
pngs[-3]

TypeError: expected str, bytes or os.PathLike object, not list